## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
# requests 라이브러리 설치여부 확인
%pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: c:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract
Note: you may need to restart the kernel to use updated packages.


In [2]:
# beautifulsoup4 라이브러리 설치여부 확인
%pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: c:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert
Note: you may need to restart the kernel to use updated packages.


In [4]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [6]:
# requests, bs4 버전 확인하기
print(f'requests 버전 {requests.__version__}')
print(f'beautifulsoup4 버전 {bs4.__version__}')

requests 버전 2.32.3
beautifulsoup4 버전 4.12.3


### 1. 뉴스 제목 추출하기

In [50]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}
# 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# url2 = 'https://news.naver.com/section/105'

# 요청 헤더 설정 : 브라우저 정보 (사람처럼 보이게 하려고 설정하는 것)
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers= req_header)
print(res)
print(type(res))
print(res.status_code, res.ok)

# 응답(response)이 OK 이면
if res.ok:
    # 응답 (response)에서 text 추출 - 마우스 우클릭 source 보기
    html = res.text
    # print(html)
    # BeautifulSoup 객체 생성  
    soup = BeautifulSoup(html,'html.parser')
    # print(soup)
    # CSS 선택자
    # <div class='sa_text'><a href=''></a></div>
    print(len(soup.select("div.sa_text a[href*='mnews/article']")))
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    a_tag_list = soup.select("div.sa_text a[href*='mnews/article']")
    print(type(a_tag_list)) # [<a href="http://">뉴스제목</a>]: ResultSet 객체, <a></a>: Tag 객체
    # print(a_tag_list) #[<a></a>, <a></a]

    # <a> 태그 리스트 순회하기 
    for a_tag in a_tag_list:
        # print(type(a_tag))
        # print(a_tag.text.strip())
        title = a_tag.text.strip()
        #href 속성의 값을 가져오기
        link = a_tag['href']
        print(link)
        print(title)
   
else:
    # 응답(response)이 Error 이면 status code 출력  
    print(f'에러 코드 = {res.status_code}')  



https://news.naver.com/section/105
<Response [200]>
<class 'requests.models.Response'>
200 True
82
<class 'bs4.element.ResultSet'>
https://n.news.naver.com/mnews/article/032/0003361960
악성 스팸문자, 발송단계에서 ‘X-ray’로 차단한다
https://n.news.naver.com/mnews/article/comment/032/0003361960

https://n.news.naver.com/mnews/article/015/0005116490
"영락없는 자녀 목소린데"…AI 보이스피싱, AI로 방지
https://n.news.naver.com/mnews/article/comment/015/0005116490

https://n.news.naver.com/mnews/article/015/0005116019
뤼튼·시나몬·무빈·스튜디오랩…AI 스타트업에 자금 몰린다
https://n.news.naver.com/mnews/article/comment/015/0005116019

https://n.news.naver.com/mnews/article/030/0003301410
1분기 효자는 단연 '갤S25'…4년 만에 4조원 이익 돌파[이슈플러스]
https://n.news.naver.com/mnews/article/comment/030/0003301410

https://n.news.naver.com/mnews/article/277/0005574594
"가구 배치, 옷 코디도 도와준다"…제미나이 라이브 갤S25 적용
https://n.news.naver.com/mnews/article/comment/277/0005574594

https://n.news.naver.com/mnews/article/092/0002369859
순이익 3분의 1인데…MBK, '오스템임플란트' 고배당 논란
https://n.news.naver.com

### 1.1 뉴스제목 추출하는 함수 선언하기

In [44]:
import requests
from bs4 import BeautifulSoup

section_dict = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}


def print_news(sid):  # print_new(103,'생활/문화')
    section = section_dict[sid]
    req_param = {"sid": sid}
    #
    # req_param 안 쓰는 경우엔
    # url = f"https://news.naver.com/section/{sid}"
    url = "https://news.naver.com/section/{sid}".format(**req_param)
    print(f"====> {url} {section} 뉴스 <====")

    # 요청 헤더 설정 : 브라우저 정보 (사람처럼 보이게 하려고 설정하는 것)
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    # requests 의 get() 함수 호출하기
    res = requests.get(url, headers=req_header)
    print(res)
    print(type(res))
    print(res.status_code, res.ok)

    # 응답(response)이 OK 이면
    if res.ok:
        # 응답 (response)에서 text 추출 - 마우스 우클릭 source 보기
        html = res.text
        # print(html)
        # BeautifulSoup 객체 생성  
        soup = BeautifulSoup(html,'html.parser')
        # print(soup)
        # CSS 선택자
        # <div class='sa_text'><a href=''></a></div>
        print(len(soup.select("div.sa_text a[href*='mnews/article']")))
        # print(soup.select("div.sa_text a[href*='mnews/article']"))
        a_tag_list = soup.select("div.sa_text a[href*='mnews/article']")
        print(type(a_tag_list)) # [<a href="http://">뉴스제목</a>]: ResultSet 객체, <a></a>: Tag 객체
        # print(a_tag_list) #[<a></a>, <a></a]

        # <a> 태그 리스트 순회하기 
        for a_tag in a_tag_list:
            # print(type(a_tag))
            # print(a_tag.text.strip())
            title = a_tag.text.strip()
            #href 속성의 값을 가져오기
            link = a_tag['href']
            print(link, title)
   
    else:
        # 응답(response)이 Error 이면 status code 출력  
        print(f'에러 코드 = {res.status_code}') 
   

In [46]:
print_news(104)

====> https://news.naver.com/section/104 세계 뉴스 <====
<Response [200]>
<class 'requests.models.Response'>
200 True
90
<class 'bs4.element.ResultSet'>
https://n.news.naver.com/mnews/article/082/0001320137 시총 1위 애플 주가 3일간 19% 하락…“상호관세 아이폰에는 재앙”
https://n.news.naver.com/mnews/article/comment/082/0001320137 
https://n.news.naver.com/mnews/article/001/0015316040 트럼프, 中에 50% 추가관세 위협…"다른 나라와는 즉시 협상"(종합)
https://n.news.naver.com/mnews/article/comment/001/0015316040 
https://n.news.naver.com/mnews/article/016/0002454342 상어출몰에 성폭력까지…신혼여행 성지 ‘이 섬’ 어쩌다 이지경
https://n.news.naver.com/mnews/article/comment/016/0002454342 
https://n.news.naver.com/mnews/article/214/0001416804 중국 관영매체 "미국 농산품 관세 대폭 인상·가금육 수입 금지 검토"
https://n.news.naver.com/mnews/article/comment/214/0001416804 
https://n.news.naver.com/mnews/article/052/0002177110 '상호관세 연구' 경제학자 "트럼프, 내 논문 완전히 잘못 해석"
https://n.news.naver.com/mnews/article/comment/052/0002177110 
https://n.news.naver.com/mnews/article/020/0003626815 US스틸 결국 일본제철에 인수되나…트럼프 

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [ ]:
import requests
import os

req_header = {
    'referer':''
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg'
]

for img_url in img_urls:
    pass
    # requests 의 get(url, headers) 함수 호출하기 
        
        # binary 응답 데이터 가져오기
        
        # url에서 파일명만 추출하기
        
        # binday data를 file에 write하기


200
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg(299,319 bytes)
200
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg(163,333 bytes)
200
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg(182,530 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.

200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [2]:
# 캡차 키 발급 요청


In [1]:
# 캡차 이미지 요청


In [3]:
#  사용자 입력값 검증 요청



* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
